In [1]:
!pip install neo4j
!pip install graphdatascience
!pip install psycopg2

In [1]:
import pandas as pd
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience
import psycopg2

# Postgres Database connection parameters
hostname = 'localhost'
database = 'project'
username = 'postgres'
password = 'password'
port = '5432'  


conn = psycopg2.connect(
    host=hostname,
    database=database,
    user=username,
    password=password,
    port=port
)
cur = conn.cursor()


uri = "neo4j://localhost:7687"
username = "neo4j"
password = "password"

In [2]:
driver = GraphDatabase.driver(uri, auth=(username, password))
gds = GraphDataScience(uri, auth=(username, password))
session =  driver.session()

In [3]:
def show_result(query):
    result = session.run(query)
    return(result.data())

In [6]:
def delete_graph_if_exists(graph_name):
    try:
        existing_graph = gds.graph.get(graph_name)
        if existing_graph is not None:
            gds.graph.drop(graph_name)
            print(f"Graph '{graph_name}' dropped.")
    except Exception as e:
        print(f"Graph '{graph_name}' does not exist or could not be retrieved. Nothing to delete.")

In [4]:
def create_movie_projection(graph_name="movie_similarity_graph", directed=False):
    """Creates a movie similarity projection based on shared cast & crew."""
    
    delete_graph_if_exists(graph_name)
    
    if not directed:
        projection, project_result = gds.graph.project(graph_name, ["Person", "Movie"], {
        'self': {'orientation': 'UNDIRECTED'},
        'director': {'orientation': 'UNDIRECTED'},
        'producer': {'orientation': 'UNDIRECTED'},
        'cinematographer': {'orientation': 'UNDIRECTED'},
        'writer': {'orientation': 'UNDIRECTED'},
        'composer': {'orientation': 'UNDIRECTED'},
        'editor': {'orientation': 'UNDIRECTED'},
        'actor': {'orientation': 'UNDIRECTED'},
        'actress': {'orientation': 'UNDIRECTED'},
        'archive_footage': {'orientation': 'UNDIRECTED'},
        'production_designer': {'orientation': 'UNDIRECTED'},
        'casting_director': {'orientation': 'UNDIRECTED'},
        'archive_sound': {'orientation': 'UNDIRECTED'}
    })
        
    else:
        projection, project_result = gds.graph.project(graph_name, ["Person", "Movie"], ['self',
         'director',
         'producer',
         'cinematographer',
         'writer',
         'composer',
         'editor',
         'actor',
         'actress',
         'archive_footage',
         'production_designer',
         'casting_director',
         'archive_sound'])
    return (graph_name, projection, project_result)

In [7]:
ms_graph_name, ms_projection, _ = create_movie_projection('movie_similarity_graph')

Graph 'movie_similarity_graph' does not exist or could not be retrieved. Nothing to delete.


In [9]:
def get_top_similar_movies(movie_name='Leaving the Factory', graph_name="movie_similarity_graph", top_n=5):
    """Fetches the top similar movies with their ratings for a given movie."""
    
    #1. Query neo4j for the top_n movies 
    query = f"""
    MATCH (m:Movie {{primaryTitle: $movie_name}})
    WITH id(m) AS movieId
    CALL gds.nodeSimilarity.stream('{graph_name}', {{
        similarityCutoff: 0.1,
        topK: {top_n}
    }})
    YIELD node1, node2, similarity
    WHERE node1 = movieId OR node2 = movieId
    WITH CASE WHEN node1 = movieId THEN node2 ELSE node1 END AS similarMovieId, similarity
    MATCH (rec:Movie) WHERE id(rec) = similarMovieId
    RETURN rec.tconst as tconst, similarity AS similarity_score
    ORDER BY similarity DESC
    LIMIT {top_n}
    """
    
    results = session.run(query, movie_name=movie_name)
    
    records = [{
        "tconst" : record['tconst'],
        "similarity_score": record["similarity_score"]
    } for record in results]
    
    intermediate = pd.DataFrame(records)

    tconst_values = intermediate['tconst'].tolist()  
    similarity_scores = intermediate['similarity_score'].tolist()

    
    #2. Query Postgres for the corresponding details of movies
    query = f"""
    SELECT distinct d.primarytitle, d.genres, r.averagerating, t.similarity_score
    FROM (SELECT unnest(array{tconst_values}) AS tconst, unnest(array{similarity_scores}) AS similarity_score) t
    LEFT JOIN title_ratings r ON t.tconst = r.tconst
    LEFT JOIN title_basics d ON t.tconst = d.tconst
    """
    
    cur.execute(query)
    details = cur.fetchall()
    
    columns = ["title", "genres", "average_rating", "similarity_score"]
    records = []
    for row in details:
        records.append({
            "title": row[0],
            "genres": row[1],
            "average_rating": row[2],
            "similarity_score": row[3]
        })
    
    df = pd.DataFrame(records, columns=columns)
    return(intermediate,df)

In [10]:
intermediate, recommendations = get_top_similar_movies('The Arrival of a Train',ms_graph_name)
recommendations.head()

,title,genres,average_rating,similarity_score
0,Baby's Meal,"Documentary,Short",5.9,0.2727272727272727
1,Bataille de neige,"Comedy,Documentary,Short",6.7,0.2
2,Boat Leaving the Port,"Documentary,Short",5.9,0.3
3,Demolition of a Wall,"Documentary,Short",6.4,0.3
4,Leaving Jerusalem by Railway,"Documentary,Short",6.2,0.2727272727272727


In [11]:
intermediate.head()

,tconst,similarity_score
0,tt0000070,0.300000
1,tt0000016,0.300000
2,tt0000089,0.272727
3,tt0000029,0.272727
4,tt0000041,0.200000
